In [ ]:
import os
import time
import tonic
import tonic.transforms as transforms
import numpy as np
from utils import write_dataset, custom_sampler

## Parameters

In [ ]:
# dataset parameters
dataset_name = 'POKERDVS' # name of dataset: POKERDVS -- NMNIST -- NCARS -- DVSGesture
save_as_double = True # make the timesurface dataset either double or float 
download_dataset = True # downloads the datasets before parsing
subsample = 100 # take a sample of the dataset

# timesurface parameters
decay='exp'
time_constant = 80e3 # decay time constant for time surfaces
surface_dimensions = [9,9] # surface dimensions not necessarily square

# denoising parameters
denoise = False # denoising that removes solitary events
denoise_filter = 1e3 # decrease time filter to remove more events if denoising is turned on
add_noise = False
refractory = True # add refractory period
refractory_period = 0.5

# spatial histogram for better classification
split_grid = False # save location of timesurfaces in a grid
K = 10 # size of each cell in grid

# plotting parameters
plot_dictionary = False
plot_covariance = False

### Getting dataset

In [ ]:
transform = transforms.Compose([transforms.ToTimesurface(surface_dimensions=surface_dimensions, tau=time_constant, decay=decay, merge_polarities=True)])
    
if denoise: transform.transforms = [transforms.Denoise(time_filter=denoise_filter)] + transform.transforms 
if refractory: transform.transforms = [transforms.RefractoryPeriod(refractory_period=refractory_period)] + transform.transforms 
    
if dataset_name == 'NCARS': # 304 x 240
    train_set = tonic.datasets.NCARS(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.NCARS(save_to='./data', train=False, download=download_dataset)
if dataset_name == 'POKERDVS': # 35 x 35
    train_set = tonic.datasets.POKERDVS(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.POKERDVS(save_to='./data', train=False, download=download_dataset)
elif dataset_name == "DVSGesture": # 128 x 128
    train_set = tonic.datasets.DVSGesture(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.DVSGesture(save_to='./data', train=False, download=download_dataset)
elif dataset_name == 'NMNIST': # 34 x 34
    train_set = tonic.datasets.NMNIST(save_to='./data/nmnist', train=True, download=download_dataset)
    test_set = tonic.datasets.NMNIST(save_to='./data/nmnist', train=False, download=download_dataset)

train_folder = decay + '/' + dataset_name.lower()
train_folder = train_folder + "_train/" if not split_grid else train_folder + "_train_sp%s/" % K

test_folder = decay + '/' + dataset_name.lower()
test_folder = test_folder + "_test/" if not split_grid else test_folder + "_test_sp%s/" % K

### Generating indices (to shuffle dataset and/or create a subset)

In [ ]:
train_index = np.arange(len(train_set))
np.random.shuffle(train_index)

test_index = np.arange(len(test_set))
np.random.shuffle(test_index)

if subsample > 0 and subsample < 100:
    print("Taking %s%% of the dataset" % subsample)
    
    # calculate number of samples we want to take
    train_samples = np.ceil((subsample * len(train_set)) / 100).astype(int)
    test_samples = np.ceil((subsample * len(test_set)) / 100).astype(int)
    
    # choosing indices of the subset
    train_index = train_index[:train_samples]
    test_index = test_index[:test_samples]

### Extracting features

In [ ]:
train_surf_shapes, train_cell_index = write_dataset(train_set, train_index, train_folder, transform, surface_dimensions, save_as_double, add_noise, split_grid, K)
test_surf_shapes, test_cell_index = write_dataset(test_set, test_index, test_folder, transform, surface_dimensions, save_as_double, add_noise, split_grid, K)

tr_Total = sum([shape[1] for shape in train_surf_shapes]) # number of training data points
te_Total = sum([shape[1] for shape in test_surf_shapes]) # number of test data points
D = test_surf_shapes[0][-1] # dimensions
C = test_surf_shapes[0][2] # number of channels

np.savetxt(train_folder+'/header.txt', np.array([tr_Total, 1, C, D, train_cell_index]), delimiter=',', fmt='%d')
np.savetxt(test_folder+'/header.txt', np.array([te_Total, 1, C, D, test_cell_index]), delimiter=',', fmt='%d')

# save dataset shuffle indices
np.savetxt(train_folder+'/indices.txt', train_index, fmt='%i')
np.savetxt(test_folder+'/indices.txt', test_index, fmt='%i')

### Dictionary of timesurfaces

In [ ]:
if plot_dictionary:
    import matplotlib.pyplot as plt
    timesurfaces = np.load(test_folder+'data/5.npy')[0]
    timesurfaces = timesurfaces.reshape((timesurfaces.shape[0],timesurfaces.shape[1],int(np.sqrt(timesurfaces.shape[2])),int(np.sqrt(timesurfaces.shape[2]))))
    rows = 10
    cols = 10
    fig, axes = plt.subplots(rows,cols, figsize=(8,8))
    for i,ax in enumerate(axes.flat):
        ax.imshow(timesurfaces[i+2500,0], cmap='hot')
        ax.axis('off')
    fig.savefig(test_folder+'/dic_timesurfaces.pdf')

### covariances

In [ ]:
if plot_covariance:
    import seaborn as sn
    import matplotlib.pyplot as plt

    dataset_ts = np.squeeze(np.concatenate([np.load(test_folder+"data/"+f)[0] for f in os.listdir(test_folder+"data/") if f.endswith(".npy")]))
    dataset_ts = np.delete(dataset_ts, int(np.floor(dataset_ts[0].shape[0] / 2)), axis=1)
    covariance = np.corrcoef(dataset_ts.T)
    sn.heatmap(covariance)
    plt.show()